# CoNaLA

In [1]:
import json
train_data=[]
test_data=[]
with open('conala-corpus/conala-train.json') as f:
    data = json.load(f)
    for item in data:
        intent = item['rewritten_intent']
        if intent is None:
            intent = item['intent']
        code = item['snippet']
        train_data.append((intent, code))
with open('conala-corpus/conala-test.json') as f:
    data = json.load(f)
    for item in data:
        intent = item['rewritten_intent']
        if intent is None:
            intent = item['intent']
        code = item['snippet']
        test_data.append((intent, code))
print(f'loaded: {len(train_data)}, {len(test_data)}')


loaded: 2379, 500


In [7]:
import csv
with open('conala6_train.tsv', 'w') as f:
    w = csv.writer(f, delimiter='\t')
    for intent, code in train_data:
        code=code.replace('\n', '<nl>').replace('\t', '    ')
        w.writerow((intent, code))
with open('conala6_test.tsv', 'w') as f:
    w = csv.writer(f, delimiter='\t')
    for intent, code in test_data:
        code=code.replace('\n', '<nl>').replace('\t', '    ')
        w.writerow((intent, code))

In [4]:
import string
import re


QUOTED_STRING_RE = re.compile(
    r"(?P<quote>['`\"])(?P<string>.*?)(?<!\\)(?P=quote)")


def parse_quoted_string(search_string):
    return QUOTED_STRING_RE.findall(search_string)

def parameterized(s, t, fmt='{}', A=string.ascii_uppercase):
    quotes = parse_quoted_string(s)
    t = f' {t} '
    index = 0
    d = {}
    dt = {}
    for q in quotes:
        qq = f'{q[0]}{q[1]}{q[0]}'
        var = fmt.format(A[len(d)])
        if qq in t:
            d[var] = qq
            dt[var] = qq
            t = t.replace(qq, var)
            s = s.replace(qq, var)
            continue
        if q[1] in t:
            d[var] = qq
            dt[var] = q[1]
            if q[1].isalnum():
                pat = f'([^A-Za-z0-9])({q[1]})([^A-Za-z0-9])'
                t = re.sub(pat, f'\\1{var}\\3', t)
            else:
                t = t.replace(q[1], var)
            s = s.replace(qq, var)
    return s, t.strip(), d, dt


In [5]:
import csv
with open('conala6sp_train.tsv', 'w') as f:
    w = csv.writer(f, delimiter='\t')
    for intent, code in train_data:
        code=code.replace('\n', '<nl>').replace('\t', '    ')
        intent, code, _, _ = parameterized(intent, code)
        w.writerow((intent, code))
with open('conala6sp_test.tsv', 'w') as f:
    w = csv.writer(f, delimiter='\t')
    for intent, code in test_data:
        code=code.replace('\n', '<nl>').replace('\t', '    ')
        intent, code, _, _ = parameterized(intent, code)
        w.writerow((intent, code))
